# Bellwether Data Segmentation

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Display full data
pd.set_option('display.max_columns', None, 'display.max_rows', None)
pd.options.mode.chained_assignment

# Function to check for nan values
def isnan(x):
    return x != x

In [2]:
# Names of states
state_names = ["Alaska", "Alabama", "Arkansas", "American Samoa", "Arizona", "California", "Colorado", 
               "Connecticut", "Delaware", "Florida", "Georgia", "Guam", "Hawaii", 
               "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana", "Massachusetts", 
               "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi", "Montana", "North Carolina", 
               "North Dakota", "Nebraska", "New Hampshire", "New Jersey", "New Mexico", "Nevada", "New York", "Ohio", 
               "Oklahoma", "Oregon", "Pennsylvania", "Puerto Rico", "Rhode Island", "South Carolina", "South Dakota", 
               "Tennessee", "Texas", "Utah", "Virginia", "Virgin Islands", "Vermont", "Washington", "Wisconsin", 
               "West Virginia", "Wyoming"]

In [3]:
# NOTE: to fill missing data, we append a space (' ') to the location of the missing data in the dataset

## Segment 2000 Data

### Add Election Data

In [4]:
data_prox = pd.read_csv('data/compiled_data/data_prox.csv')

In [5]:
df = pd.read_csv('data/raw_data/election_data_2000-2016.csv')
df = df[(df['party'] == 'democrat')|(df['party'] == 'republican')]
df = df[df['year'] == 2000]
df['percentage'] = df['candidatevotes']/df['totalvotes']
df = df[df['state'] != 'Hawaii']
df = df[df['state'] != 'Alaska']
df = df[df['county'] != 'Statewide writein']
df = df[df['county'] != 'Federal Precinct']
df = df[df['county'] != 'Maine UOCAVA']
df = df[df['FIPS'] != 36000] # remove Missouri Kansas City

In [6]:
df.shape

(6216, 12)

In [7]:
year = []
fips = []
state = []
county = []
dem = []
rep = []

for i in range(int(len(df['FIPS'])/2)):
    year.append(2000)
    fips.append(df.iloc[2*i, 4])
    state.append(df.iloc[2*i, 1])
    county.append(df.iloc[2*i, 3])
    dem.append(df.iloc[2*i, 11])
    rep.append(df.iloc[2*i+1, 11])

In [8]:
data = pd.DataFrame()
data['year'] = year
data['fips'] = fips
data['state'] = state
data['county'] = county
data['dem%'] = dem
data['rep%'] = rep

In [9]:
fips = []

for x in list(data['fips']):
    fips.append(int(x))

data['fips'] = fips

In [10]:
data.head()

year  fips    state   county      dem%      rep%
0  2000  1001  Alabama  Autauga  0.287192  0.696943
1  2000  1003  Alabama  Baldwin  0.247822  0.723654
2  2000  1005  Alabama  Barbour  0.499086  0.490236
3  2000  1007  Alabama     Bibb  0.381636  0.601746
4  2000  1009  Alabama   Blount  0.276915  0.704779

In [11]:
# Add a column to see if the county predicted the election

prediction = []
for i, v in enumerate(data['fips']):
    if data.iloc[i, 4] < data.iloc[i, 5]:
        prediction.append(1)
    else:
        prediction.append(0)
data['prediction'] = prediction

In [12]:
data.head()

year  fips    state   county      dem%      rep%  prediction
0  2000  1001  Alabama  Autauga  0.287192  0.696943           1
1  2000  1003  Alabama  Baldwin  0.247822  0.723654           1
2  2000  1005  Alabama  Barbour  0.499086  0.490236           0
3  2000  1007  Alabama     Bibb  0.381636  0.601746           1
4  2000  1009  Alabama   Blount  0.276915  0.704779           1

In [13]:
data.shape

(3108, 7)

In [14]:
data.iloc[289, 1] = 11
data.iloc[289, 3] = 'Total' # District of Columbia

In [15]:
# Check to see if all fips are the same in data and data_prox

print('Counties in data not in prox:')
for i, v in enumerate(list(data['fips'])):
    if v not in list(data_prox['fips']):
        print(v)
        
print('Counties in prox not in data:')
for i, v in enumerate(list(data_prox['fips'])):
    if v not in list(data['fips']):
        print(v)

Counties in data not in prox:
46113
51515
Counties in prox not in data:
8014
46102


In [16]:
# 8014 added in 2001
# 46102 is the equivalent of Oglala Lakota County, SD (46113)

In [17]:
data.iloc[2882, 3] = 'Bedford City' # not to be confused with Bedford County

In [18]:
# Not all 2000 counties are in this dataset, we add 51560
data = data.append(pd.DataFrame([[2000, 51560, 'Virginia', 'Clifton Forge City', ' ', ' ', ' ']], columns=['year', 'fips', 'state', 'county', 'dem%', 'rep%', 'prediction']), ignore_index=True)

In [19]:
data.tail() # no election data for 51560

year   fips     state              county      dem%      rep% prediction
3104  2000  56039   Wyoming               Teton  0.385294  0.522865          1
3105  2000  56041   Wyoming               Uinta  0.222432  0.737261          1
3106  2000  56043   Wyoming            Washakie  0.198963  0.774624          1
3107  2000  56045   Wyoming              Weston  0.146732  0.823856          1
3108  2000  51560  Virginia  Clifton Forge City

In [20]:
data.shape # 3109 mainland US counties in 2000

(3109, 7)

### Add Poverty and Population Data

In [21]:
p_df = pd.read_csv('data/raw_data/poverty_rate.csv', engine='python')
p_df = p_df.drop(columns=['Number'])
p_df = p_df[p_df['County'] != 'State Total']
p_df = p_df[p_df['State'] != 'Hawaii']
p_df = p_df[p_df['State'] != 'Alaska']
p_df.head()

FIPS    State     County  Poverty Rate 1960  Poverty Rate 1970  \
1  1001  Alabama  Autauga                 53.1               23.7   
2  1003  Alabama  Baldwin                 41.8               22.6   
3  1005  Alabama   Barbour                65.7               43.7   
4  1007  Alabama      Bibb                56.8               38.2   
5  1009  Alabama    Blount                45.6               24.2   

   Poverty Rate 1980  Poverty Rate 1990  Poverty Rate 2000  Poverty Rate 2010  \
1               16.6               15.7               10.9               11.6   
2               16.2               14.3               10.1               13.3   
3               30.8               25.2               26.8               26.1   
4               21.2               21.2               20.6               16.5   
5               18.0               15.3               11.7               14.7   

  Population 1960 Population 1970 Population 1980 Population 1990  \
1          18,454          24,313          32,107          34,009   
2          49,722          58,776          77,726          97,227   
3          24,975          22,424          24,533          24,785   
4          14,775          13,761          15,520          16,304   
5          25,414          26,746          36,244          38,924   

  Population 2000 Population 2010 Persons in Poverty 1960  \
1          43,377          54,255                   9,807   
2         138,148         180,970                  20,773   
3          26,239          24,530                  16,395   
4          19,852          21,794                   8,386   
5          50,499          56,993                  11,595   

  Persons in Poverty 1970 Persons in Poverty 1980 Persons in Poverty 1990  \
1                   5,758                   5,337                   5,324   
2                  13,304                  12,616                  13,947   
3                   9,788                   7,559                   6,244   
4                   5,256                   3,296                   3,454   
5                   6,462                   6,527                   5,938   

  Persons in Poverty 2000 Persons in Poverty 2010  
1                   4,738                   6,267  
2                  14,018                  24,002  
3                   7,032                   6,395  
4                   4,091                   3,594  
5                   5,930                   8,396

In [22]:
# Remove commas from numerical values

columns = ['Population 1960', 'Population 1970',
       'Population 1980', 'Population 1990', 'Population 2000',
       'Population 2010', 'Persons in Poverty 1960', 'Persons in Poverty 1970',
       'Persons in Poverty 1980', 'Persons in Poverty 1990',
       'Persons in Poverty 2000', 'Persons in Poverty 2010']

for x in range(len(columns)):
    for i, v in enumerate(p_df[columns[x]]):
        if type(v) == str and ',' in v:
            v = int(''.join(v.split(',')))
            p_df.iloc[i, x+9] = v

In [23]:
# Add poverty data

poverty = []
population = []

for i, v in enumerate(data['fips']):
    for a, b in enumerate(p_df['FIPS']):
        if b == v:
            poverty.append(list(p_df['Poverty Rate 2000'])[a])
            population.append(list(p_df['Population 2000'])[a])
            break
        elif v == 51560 and b == 51005: # WARNING: in 2001 Clifton Forge City (51560) was added to Alleghany County (51005), but there is no data for 51560 so we will use the data from 51005 instead
            poverty.append(list(p_df['Poverty Rate 2000'])[a])
            population.append(list(p_df['Population 2000'])[a])
            break

In [24]:
len(poverty)

3109

In [25]:
data['poverty_rate'] = poverty
data['population'] = population

In [26]:
data.tail()

year   fips     state              county      dem%      rep%  \
3104  2000  56039   Wyoming               Teton  0.385294  0.522865   
3105  2000  56041   Wyoming               Uinta  0.222432  0.737261   
3106  2000  56043   Wyoming            Washakie  0.198963  0.774624   
3107  2000  56045   Wyoming              Weston  0.146732  0.823856   
3108  2000  51560  Virginia  Clifton Forge City                       

     prediction  poverty_rate population  
3104          1           6.0      18121  
3105          1           9.9      19360  
3106          1          14.1       8091  
3107          1           9.9       6366  
3108                      7.1      12682

In [27]:
# Check to see if the fips are the same in both datasets

print('Counties in data not in p_df:')
for i, v in enumerate(list(data['fips'])):
    if v not in list(p_df['FIPS']):
        print(v)

print('Counties in p_df not in data:')
for i, v in enumerate(list(p_df['FIPS'])):
    if v not in list(data['fips']):
        print(v)

Counties in data not in p_df:
51560
Counties in p_df not in data:
8014


### Add Unemployment Rate

In [28]:
u_df = pd.read_csv('data/raw_data/unemployment_rate.csv')
u_df.drop([0, 1, 2])

# Rename & Drop Columns

n_df = pd.DataFrame()
for i, v in enumerate(u_df.columns):
    n_df[str(u_df.iloc[3, i])] = u_df.iloc[4:, i]
u_df = n_df
u_df = u_df[u_df['Stabr'] != 'AK']
u_df = u_df[u_df['Stabr'] != 'HI']
u_df = u_df[u_df['Stabr'] != 'US']
u_df = u_df[u_df['Stabr'] != 'PR']
u_df = u_df[u_df['fips_txt'] != '11001']
u_df.iloc[297, 0] = 11 # District of Columbia
u_df.iloc[2417, 0] = 46113 # Oglala Lakota County, SD changed names
for state in state_names:
    u_df = u_df[u_df['area_name'] != state]

In [29]:
# convert u_df fips to integers

fips = []

for x in list(u_df['fips_txt']):
    fips.append(int(x))

u_df['fips_txt'] = fips

In [30]:
u_df.head()

fips_txt Stabr           area_name Rural_urban_continuum_code_2013  \
6       1001    AL  Autauga County, AL                               2   
7       1003    AL  Baldwin County, AL                               3   
8       1005    AL  Barbour County, AL                               6   
9       1007    AL     Bibb County, AL                               1   
10      1009    AL   Blount County, AL                               1   

   Urban_influence_code_2013 Metro_2013 Civilian_labor_force_2000  \
6                          2          1                    21,720   
7                          2          1                    69,533   
8                          6          0                    11,373   
9                          1          1                     8,565   
10                         1          1                    25,106   

   Employed_2000 Unemployed_2000 Unemployment_rate_2000  \
6         20,846             874                      4   
7         66,971           2,562                    3.7   
8         10,748             625                    5.5   
9          8,111             454                    5.3   
10        24,231             875                    3.5   

   Civilian_labor_force_2001 Employed_2001 Unemployed_2001  \
6                     21,955        21,055             900   
7                     69,161        66,195           2,966   
8                     11,250        10,412             838   
9                      9,081         8,468             613   
10                    25,305        24,393             912   

   Unemployment_rate_2001 Civilian_labor_force_2002 Employed_2002  \
6                     4.1                    22,094        21,035   
7                     4.3                    69,169        65,691   
8                     7.4                    10,971        10,125   
9                     6.8                     8,933         8,309   
10                    3.6                    25,757        24,366   

   Unemployed_2002 Unemployment_rate_2002 Civilian_labor_force_2003  \
6            1,059                    4.8                    22,604   
7            3,478                      5                    72,299   
8              846                    7.7                    10,977   
9              624                      7                     8,836   
10           1,391                    5.4                    25,900   

   Employed_2003 Unemployed_2003 Unemployment_rate_2003  \
6         21,462           1,142                    5.1   
7         68,702           3,597                      5   
8         10,196             781                    7.1   
9          8,309             527                      6   
10        24,702           1,198                    4.6   

   Civilian_labor_force_2004 Employed_2004 Unemployed_2004  \
6                     23,218        22,103           1,115   
7                     74,772        70,919           3,853   
8                     10,633         9,865             768   
9                      8,843         8,359             484   
10                    26,208        25,101           1,107   

   Unemployment_rate_2004 Civilian_labor_force_2005 Employed_2005  \
6                     4.8                    23,949        23,037   
7                     5.2                    76,804        73,743   
8                     7.2                    10,760        10,136   
9                     5.5                     8,861         8,463   
10                    4.2                    26,446        25,491   

   Unemployed_2005 Unemployment_rate_2005 Civilian_labor_force_2006  \
6              912                    3.8                    24,398   
7            3,061                      4                    79,711   
8              624                    5.8                    10,705   
9              398                    4.5                     8,850   
10             955                    3.6                    26,770   

   E

In [31]:
# Add unemployemnt data

unemployment = []

for i, v in enumerate(data['fips']):
    for a, b in enumerate(u_df['fips_txt']):
        if b == v:
            unemployment.append(list(u_df['Unemployment_rate_2000'])[a])
            break
        elif v == 51560 and b == 51005: # WARNING: in 2001 Clifton Forge City (51560) was added to Alleghany County (51005), but there is no data for 51560 so we will use the data from 51005 instead
            unemployment.append(list(u_df['Unemployment_rate_2000'])[a])
            break
        elif v == 51515:
            unemployment.append(' ') # WARNING: we are appending no data for 51515
            break

In [32]:
len(unemployment) # need 3109

3109

In [33]:
data['unemployment_rate'] = unemployment

In [34]:
# Check to see if the fips are the same in both datasets

print('Counties in data not in u_df:')
for i, v in enumerate(list(data['fips'])):
    if v not in list(u_df['fips_txt']):
        print(v)

print('Counties in u_df not in data:')
for i, v in enumerate(list(u_df['fips_txt'])):
    if v not in list(data['fips']):
        print(v)

Counties in data not in u_df:
51515
51560
Counties in u_df not in data:
8014


### Add Education Data

In [35]:
e_df = pd.read_csv('data/raw_data/education_rate.csv', engine='python')
for state in state_names:
    e_df = e_df[e_df['Area name'] != state]
e_df = e_df[e_df['State'] != 'AK']
e_df = e_df[e_df['State'] != 'HI']
e_df = e_df[e_df['State'] != 'US']
e_df = e_df[e_df['State'] != 'PR']
e_df.iloc[290, 0] = 11 # District of Columbia
e_df.iloc[2381, 0] = 46113 # Oglala Lakota County, SD changed names

In [36]:
e_df.shape # 3109 mainland counties in 2000

(3113, 47)

In [37]:
e_df.head()

FIPS Code State       Area name  2003 Rural-urban Continuum Code  \
2       1001    AL  Autauga County                              2.0   
3       1003    AL  Baldwin County                              4.0   
4       1005    AL  Barbour County                              6.0   
5       1007    AL     Bibb County                              1.0   
6       1009    AL   Blount County                              1.0   

   2003 Urban Influence Code  2013 Rural-urban Continuum Code  \
2                        2.0                              2.0   
3                        5.0                              3.0   
4                        6.0                              6.0   
5                        1.0                              1.0   
6                        1.0                              1.0   

   2013 Urban Influence Code Less than a high school diploma, 1970  \
2                        2.0                                 6,611   
3                        2.0                                18,726   
4                        6.0                                 8,120   
5                        1.0                                 5,272   
6                        1.0                                10,677   

  High school diploma only, 1970 Some college (1-3 years), 1970  \
2                          3,757                            933   
3                          8,426                          2,334   
4                          2,242                            581   
5                          1,402                            238   
6                          3,440                            626   

  Four years of college or higher, 1970  \
2                                   767   
3                                 2,038   
4                                   861   
5                                   302   
6                                   404   

   Percent of adults with less than a high school diploma, 1970  \
2                                               54.8              
3                                               59.4              
4                                               68.8              
5                                               73.1              
6                                               70.5              

   Percent of adults with a high school diploma only, 1970  \
2                                               31.1         
3                                               26.7         
4                                               19.0         
5                                               19.4         
6                                               22.7         

   Percent of adults completing some college (1-3 years), 1970  \
2                                                7.7             
3                                                7.4             
4                                                4.9             
5                                                3.3             
6                                                4.1             

   Percent of adults completing four years of college or higher, 1970  \
2                                                6.4                    
3                                                6.5                    
4                                                7.3                    
5                                                4.2                    
6                                                2.7                    

  Less than a high school diploma, 1980 High school diploma only, 1980  \
2                                 7,074                          6,145   
3                                18,125                         15,380   
4                                 7,759                          3,671   
5                                 5,254                          2,611   
6                                11,666                          7,011   

  Some college (1-3 years), 1980 Four years o

In [38]:
# Add education data

no_diploma = []
diploma_only = []
some_college = []
completed_college = []

for i, v in enumerate(data['fips']):
    for a, b in enumerate(e_df['FIPS Code']):
        if b == v:
            no_diploma.append(list(e_df['Percent of adults with less than a high school diploma, 2000'])[a])
            diploma_only.append(list(e_df['Percent of adults with a high school diploma only, 2000'])[a])
            some_college.append(list(e_df["Percent of adults completing some college or associate's degree, 2000"])[a])
            completed_college.append(list(e_df["Percent of adults with a bachelor's degree or higher, 2000"])[a])
            break

In [39]:
len(no_diploma)

3109

In [40]:
data['no_diploma'] = no_diploma
data['dimploma_only'] = diploma_only
data['some_college'] = some_college
data['completed_college'] = completed_college

In [41]:
data.tail()

year   fips     state              county      dem%      rep%  \
3104  2000  56039   Wyoming               Teton  0.385294  0.522865   
3105  2000  56041   Wyoming               Uinta  0.222432  0.737261   
3106  2000  56043   Wyoming            Washakie  0.198963  0.774624   
3107  2000  56045   Wyoming              Weston  0.146732  0.823856   
3108  2000  51560  Virginia  Clifton Forge City                       

     prediction  poverty_rate population unemployment_rate  no_diploma  \
3104          1           6.0      18121               2.5         5.3   
3105          1           9.9      19360               4.1        15.2   
3106          1          14.1       8091               4.7        14.4   
3107          1           9.9       6366               4.3        14.8   
3108                      7.1      12682               3.1        25.0   

      dimploma_only  some_college  completed_college  
3104           18.9          30.0               45.8  
3105           35.6          34.2               15.0  
3106           33.8          33.1               18.7  
3107           40.2          30.5               14.5  
3108           36.4          29.0                9.6

In [42]:
# Check to see if the fips are the same in both datasets

print('Counties in data not in e_df:')
for i, v in enumerate(list(data['fips'])):
    if v not in list(e_df['FIPS Code']):
        print(v)

print('Counties in e_df not in data:')
for i, v in enumerate(list(e_df['FIPS Code'])):
    if v not in list(data['fips']):
        print(v)

Counties in data not in e_df:
Counties in e_df not in data:
8014
11001
22000
30113


In [43]:
# 8014 added in 2001
# 11001 is a duplicate for D.C.
# 22000 is the state total for Louisiana
# 30113 is Yellowstone National Park, MT that was deleted in 1997
# 51515 is Bedford City, VA (not to be confused with Bedford County (51019)) and is a valid county in 2000
# 51560 is Clifton Forge City, VA and is a valid county in 2000

### Add Demographics Data

In [44]:
d_df = pd.read_csv('data/raw_data/demographics.csv', encoding='cp1252')

In [45]:
# Filter for 2000 mainland data

d_df1 = d_df
d_df1 = d_df1[(d_df.STNAME != 'Alaska')&(d_df.STNAME != 'Hawaii')]
d_df1 = d_df1[(d_df.YEAR == 1)&(d_df.AGEGRP == 0)] # not sure what AGE_GRP is

<ipython-input-45-15358e2f8054>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  d_df1 = d_df1[(d_df.YEAR == 1)&(d_df.AGEGRP == 0)] # not sure what AGE_GRP is


In [46]:
# no 2000 data in this dataset

In [47]:
# Add empty columns (for other years to fill)

z = ' '
data['percent_white_male'] = z
data['percent_white_female'] = z
data['percent_black_male'] = z
data['percent_black_female'] = z
data['percent_native_american_male'] = z
data['percent_native_american_female'] = z
data['percent_asian_male'] = z
data['percent_asian_female'] = z
data['percent_hispanic_male'] = z
data['percent_hispanic_female'] = z

In [48]:
data.head()

year  fips    state   county      dem%      rep% prediction  poverty_rate  \
0  2000  1001  Alabama  Autauga  0.287192  0.696943          1          10.9   
1  2000  1003  Alabama  Baldwin  0.247822  0.723654          1          10.1   
2  2000  1005  Alabama  Barbour  0.499086  0.490236          0          26.8   
3  2000  1007  Alabama     Bibb  0.381636  0.601746          1          20.6   
4  2000  1009  Alabama   Blount  0.276915  0.704779          1          11.7   

  population unemployment_rate  no_diploma  dimploma_only  some_college  \
0      43377                 4        21.3           33.8          26.9   
1     138148               3.7        18.0           29.6          29.3   
2      26239               5.5        35.3           32.4          21.3   
3      19852               5.3        36.8           35.7          20.4   
4      50499               3.5        29.6           36.0          24.8   

   completed_college percent_white_male percent_white_female  \
0               18.0                                           
1               23.1                                           
2               10.9                                           
3                7.1                                           
4                9.6                                           

  percent_black_male percent_black_female percent_native_american_male  \
0                                                                        
1                                                                        
2                                                                        
3                                                                        
4                                                                        

  percent_native_american_female percent_asian_male percent_asian_female  \
0                                                                          
1                                                                          
2                                                                          
3                                                                          
4                                                                          

  percent_hispanic_male percent_hispanic_female  
0                                                
1                                                
2                                                
3                                                
4

### Add COVID-19 Data

In [49]:
# Add empty columns (for other years to fill)

z = ' '
data['cumulative_confirmed_cases'] = z
data['cumulative_deaths'] = z

In [50]:
data.head()

year  fips    state   county      dem%      rep% prediction  poverty_rate  \
0  2000  1001  Alabama  Autauga  0.287192  0.696943          1          10.9   
1  2000  1003  Alabama  Baldwin  0.247822  0.723654          1          10.1   
2  2000  1005  Alabama  Barbour  0.499086  0.490236          0          26.8   
3  2000  1007  Alabama     Bibb  0.381636  0.601746          1          20.6   
4  2000  1009  Alabama   Blount  0.276915  0.704779          1          11.7   

  population unemployment_rate  no_diploma  dimploma_only  some_college  \
0      43377                 4        21.3           33.8          26.9   
1     138148               3.7        18.0           29.6          29.3   
2      26239               5.5        35.3           32.4          21.3   
3      19852               5.3        36.8           35.7          20.4   
4      50499               3.5        29.6           36.0          24.8   

   completed_college percent_white_male percent_white_female  \
0               18.0                                           
1               23.1                                           
2               10.9                                           
3                7.1                                           
4                9.6                                           

  percent_black_male percent_black_female percent_native_american_male  \
0                                                                        
1                                                                        
2                                                                        
3                                                                        
4                                                                        

  percent_native_american_female percent_asian_male percent_asian_female  \
0                                                                          
1                                                                          
2                                                                          
3                                                                          
4                                                                          

  percent_hispanic_male percent_hispanic_female cumulative_confirmed_cases  \
0                                                                            
1                                                                            
2                                                                            
3                                                                            
4                                                                            

  cumulative_deaths  
0                    
1                    
2                    
3                    
4

In [51]:
data.to_csv('data/segmented_data/data_2000.csv') # 2000 dataset